In [ ]:


import os
os.chdir(r'C:\code\bedford-ubid')

import plotly.io as pio
import contextily as ctx  # For basemaps
from scipy import stats
from pathlib import Path

from urllib.parse import urlencode

import plotly.graph_objects as go
import plotly.express as px
from folium import plugins
import folium
import seaborn as sns
import matplotlib.pyplot as plt
from shapely.geometry import Point, Polygon
import geopandas as gpd
import pandas as pd
import numpy as np
import warnings
import pyproj
from pyproj import CRS
import xlwings as xw
from pymodule.folium_plots import create_folium_polygon

warnings.filterwarnings('ignore')
plt.style.use('default')
sns.set_palette("husl")


# Set the default template to "plotly_white"

pio.templates.default = "simple_white"
%load_ext autoreload
%autoreload 2

### buildings data from ms-buildings

In [ ]:
buildings_proj = pd.read_pickle('./dataprocess/buildings_projected.pickle')

### xl data

In [ ]:

xlbook = xw.Book('./datasets/Multifamily Building Stock.xlsx')
cbl = pd.DataFrame(xlbook.sheets['Sheet1'].used_range.value)
cbl.columns = cbl.iloc[0, :]
cbl = cbl.iloc[1:, :]

### westchester tax parcel

In [ ]:
parcels_proj = pd.read_pickle('./dataprocess/parcels_projected.pickle')
parcels_proj = parcels_proj.loc[~parcels_proj.SBL.isna()]
assert len(parcels_proj['SBL'].unique()) == len(parcels_proj)
parcels_proj = parcels_proj.set_index('SBL', drop=False)

### match westchester tax :- ms-buildings

In [ ]:
def find_best_parcel_hybrid(building_geom, parcels_gdf,
                            min_overlap_pct=0.8,
                            buffer_distance=10.0,
                            prefer_area_match=True):
    """
    Try area overlap first, then buffered centroid method
    """
    building_area = building_geom.area
    building_centroid = building_geom.centroid

    potential_matches = list(parcels_gdf.sindex.intersection(building_geom.bounds))

    best_match = None
    best_score = 0
    match_method = None

    for idx in potential_matches:
        parcel_geom = parcels_gdf.iloc[idx].geometry
        parcel_idx = parcels_gdf.iloc[idx].name

        try:
            # Method 1: Area overlap
            intersection = building_geom.intersection(parcel_geom)
            if not intersection.is_empty:
                overlap_pct = intersection.area / building_area
                if overlap_pct >= min_overlap_pct:
                    if overlap_pct > best_score:
                        best_score = overlap_pct
                        best_match = parcel_idx
                        match_method = f'area_{overlap_pct:.2f}'

            # Method 2: Buffered containment (if no good area match)
            if best_match is None or not prefer_area_match:
                if parcel_geom.buffer(buffer_distance).contains(building_centroid):
                    # Calculate a "score" based on distance to parcel edge
                    distance_to_edge = building_centroid.distance(parcel_geom.boundary)
                    score = 1.0 / (1.0 + distance_to_edge)  # Closer = higher score

                    if score > best_score or best_match is None:
                        best_score = score
                        best_match = parcel_idx
                        match_method = f'buffer_{distance_to_edge:.1f}ft'

        except Exception as e:
            continue

    return best_match, best_score, match_method


# Apply hybrid approach
results = []
for idx, building in buildings_proj.iterrows():
    parcel_idx, score, method = find_best_parcel_hybrid(
        building.geometry,
        parcels_proj,
        min_overlap_pct=0.80,
        buffer_distance=15.0
    )

    results.append({
        'building_idx': idx,
        'parcel_idx': parcel_idx,
        'match_score': score,
        'match_method': method
    })

matches_df = pd.DataFrame(results)

# Summary
method_counts = matches_df['match_method'].value_counts()
print("Match methods used:")
print(method_counts)
print(f"\nTotal matches: {len(matches_df[matches_df['parcel_idx'].notna()])}")



Match methods used:
match_method
area_1.00       6494
area_0.99         27
area_0.98         23
area_0.96         19
area_0.88         13
                ... 
buffer_3.8ft       1
buffer_6.1ft       1
buffer_3.2ft       1
area_0.81          1
buffer_7.5ft       1
Name: count, Length: 82, dtype: int64

Total matches: 6748


In [ ]:
# Add parcel information to buildings
building_parcel_match = buildings_proj.merge(
    matches_df,
    left_index=True,
    right_on='building_idx',
    how='left'
)

# # Merge in parcel attributes
building_parcel_match = building_parcel_match.merge(
    parcels_proj,
    left_on='parcel_idx',
    right_index=True,
    how='left',
    suffixes=('', '_parcel')
)

building_parcel_match.to_pickle('./dataprocess/building_parcel_match.pickle')


### left merge on cbl

In [ ]:
cbl_match = building_parcel_match.loc[building_parcel_match['PRINT_KEY'].isin(cbl['Parcel ID'])].copy()

cbl_merge = pd.merge(
    left=cbl,
    right=building_parcel_match,
    left_on='Parcel ID',
    right_on='PRINT_KEY',
    how='left'
)


### qc missing cbl ids

In [ ]:
missing_parcel_ids = [x for x in cbl['Parcel ID'].unique() if x not in building_parcel_match['PRINT_KEY'].unique()]

missing_parcel_df = parcels_proj.loc[parcels_proj['PRINT_KEY'].isin(missing_parcel_ids)].copy()

In [ ]:

"""REFORMAT CRS DATASETS"""

# create duplicate of cbl_merge (one for parcel outline, one for building outline) -- assigning crs requires single 'geometry' column
parcel_geo = cbl_merge.copy()
parcel_geo['geometry'] = parcel_geo['geometry_parcel']
parcel_geo['label'] = 'parcel'
parcel_geo = gpd.GeoDataFrame(parcel_geo.drop('geometry_parcel', axis=1), crs='EPSG:2262')
parcel_geo = parcel_geo.drop_duplicates(subset=['Parcel ID'], keep='first')
parcel_geo = parcel_geo.to_crs('EPSG:4326')




building_geo = cbl_merge.copy().drop('geometry_parcel', axis=1)
building_geo = gpd.GeoDataFrame(building_geo, crs='EPSG:2262')
building_geo['label'] = 'building'
building_geo = building_geo.to_crs('EPSG:4326')



missing_parcel_geo = missing_parcel_df.copy()
missing_parcel_geo = gpd.GeoDataFrame(missing_parcel_geo, crs='EPSG:2262')
missing_parcel_geo['label'] = 'missing_parcel'
missing_parcel_geo = missing_parcel_geo.to_crs('EPSG:4326')


### check match - (westchester&msbuildings) - (cbl)


In [ ]:

"""MAP SETUP"""
center_lat = parcel_geo.geometry.centroid.y.iloc[0]
center_lon = parcel_geo.geometry.centroid.x.iloc[0]

fmap = folium.Map(location=[center_lat, center_lon], zoom_start=15)

meta_fields = ['label', 'PARCEL_ADD', 'MAIL_ADDR', 'PRINT_KEY']
meta_aliases = meta_fields

"""CREATE BUILDING POLYGONS"""
building_polygons = folium.GeoJson(
    building_geo,
    style_function=lambda feature: {
        'fillColor': 'blue',
        'color': 'blue',
        'weight': 2,
        'fillOpacity': 0.2,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=meta_fields,
        aliases=meta_aliases,
        sticky=False,
        labels=True
    ),
    popup=folium.GeoJsonPopup(
        fields=meta_fields,
        aliases=meta_aliases,
        labels=True
    ),
    highlight_function=lambda x: {
        'fillOpacity': 0.8,  # Hover opacity
    },
).add_to(fmap)


"""CREATE PARCEL POLYGONS"""
parcel_polygons = folium.GeoJson(
    parcel_geo,
    style_function=lambda feature: {
        'fillColor': 'green',
        'color': 'green',
        'weight': 3,
        'fillOpacity': 0.1,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=meta_fields,
        aliases=meta_aliases,
        sticky=False,
        labels=True
    ),
    popup=folium.GeoJsonPopup(
        fields=meta_fields,
        aliases=meta_aliases,
        labels=True
    ),
    highlight_function=lambda x: {
        'fillOpacity': 0.8,  # Hover opacity
    },
).add_to(fmap)



"""CREATE MISSING PARCEL POLYGONS"""
parcel_polygons = folium.GeoJson(
    missing_parcel_geo,
    style_function=lambda feature: {
        'fillColor': 'orange',
        'color': 'orange',
        'weight': 3,
        'fillOpacity': 0.5,
    },
    tooltip=folium.GeoJsonTooltip(
        fields=meta_fields,
        aliases=meta_aliases,
        sticky=False,
        labels=True
    ),
    popup=folium.GeoJsonPopup(
        fields=meta_fields,
        aliases=meta_aliases,
        labels=True
    ),
    highlight_function=lambda x: {
        'fillOpacity': 0.8,  # Hover opacity
    },
).add_to(fmap)


"""OUTPUT DATA"""

# parcel_polygons.add_to(fmap)
# building_polygons.add_to(fmap)
# building_polygons.add_to(fmap)

# fmap.save('html_out/polygon_map.html')


'OUTPUT DATA'

In [ ]:
# missing_parcel_geo.to_pickle('./dataprocess/missing_parcels.pickle')

In [ ]:

# [x for x in cbl['Parcel ID'].unique() if x not in building_parcel_match['PRINT_KEY'].unique()]



# missing_parcel_geo = gpd.GeoDataFrame(missing_parcels, crs='EPSG:2262')
# missing_parcel_geo = missing_parcel_geo.to_crs('EPSG:4326')

# missing_parcel_geo['label'] = 'missing_parcel_geo'

# center_lat = missing_parcel_geo.geometry.centroid.y.iloc[0]
# center_lon = missing_parcel_geo.geometry.centroid.x.iloc[0]

# meta_fields = ['label']
# meta_aliases = ['label']

# fmap = folium.Map(location=[center_lat, center_lon], zoom_start=15)

# unmatched_parcel_polygons = folium.GeoJson(
#     missing_parcel_geo,
#     style_function=lambda feature: {
#         'fillColor': 'green',
#         'color': 'green',
#         'weight': 3,
#         'fillOpacity': 0.1,
#     },
#     tooltip=folium.GeoJsonTooltip(
#         fields=meta_fields,
#         aliases=meta_aliases,
#         sticky=False,
#         labels=True
#     ),
#     popup=folium.GeoJsonPopup(
#         fields=meta_fields,
#         aliases=meta_aliases,
#         labels=True
#     ),
#     highlight_function=lambda x: {
#         'fillOpacity': 0.8,  # Hover opacity
#     },
# )


# """OUTPUT DATA"""

# unmatched_parcel_polygons.add_to(fmap)

### qc / stats

In [ ]:

# print(f'{len(matched_parcels)} matched parcels')
# print(f'{len(unmatched_parcels)} unmatched parcels')
# print(f'{len(more_than_one_building)} of {len(buildings_per_parcel)} parcels have more than one building')

# print(f'{len(outlier_buildings)} of {len(buildings_per_parcel)} parcels > 5 buildings.')
# print('highest 5:')
# print(outlier_buildings.head())
# print('lowest 5:')
# print(outlier_buildings.tail())

# fig = px.histogram(buildings_per_parcel.values)

# fig.update_layout(
#     title='distribution, buildings per parcel - BEDFORD',
#     width=600,
#     height=400
# )

# fig.show()